This code will generate the feature vector, using sub grids to calculate the overlap

This code will calculate today's met and ndays out's met in today's polygon

In [1]:
import pandas as pd
#pd.set_option('display.max_rows', None)
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import cartopy.crs as ccrs
import cartopy
import numpy as np
import netCDF4 as nc
np.set_printoptions(threshold=100000)
from shapely.geometry import Polygon, Point, MultiPoint
from shapely.ops import cascaded_union
from datetime import datetime, timedelta
import time
import warnings
warnings.filterwarnings('ignore')
from joblib import Parallel, delayed

##  Step 0. Throw out fires that have 0 viirs detects on any day. (fires are already filtered to be in the right lat/lon range

In [2]:
# load in the polygons and attributes
fire_polygons = gpd.read_file('fire_polygons_only.geojson')
fire_attributes = pd.read_csv('fire_polygons_attributes.csv')
incidents = pd.read_csv('unique_fires.csv')
print(len(fire_polygons), len(fire_attributes), len(incidents))

#throw out fires that have 0 days with detects
daily_drop_inds = []
incident_drop_inds = []
for item in incidents['Incident Number']:
    pts = fire_attributes['Number of VIIRS points'].values[fire_attributes['Incident Number']==item]
    if len(pts[pts==0])==len(pts):
        #print(item, 'oops all zeros')
        daily_drop_inds = np.append(daily_drop_inds, np.where(fire_attributes['Incident Number']==item)[0])
        incident_drop_inds = np.append(incident_drop_inds, np.where(incidents['Incident Number']==item)[0])

daily_drop_inds = [int(p) for p in daily_drop_inds]
#print(daily_drop_inds)

fire_attributes = fire_attributes.drop(labels= daily_drop_inds)
fire_attributes = fire_attributes.drop(labels='Unnamed: 0', axis=1)
fire_polygons = fire_polygons.drop(labels = daily_drop_inds)
incidents = incidents.drop(labels = incident_drop_inds)
print(len(fire_polygons), len(fire_attributes), len(incidents))

fire_attributes

22280 22280 1354
19137 19137 1036


,Incident Number,Fire Name,Current Day,Lat Fire,Lon Fire,Number of VIIRS points
5,10662684.0,PAINTED WAGON,2019-04-08,33.786944,-112.753333,3.0
6,10662684.0,PAINTED WAGON,2019-04-09,33.786944,-112.753333,0.0
7,10662684.0,PAINTED WAGON,2019-04-10,33.786944,-112.753333,0.0
8,10663171.0,LONE MOUNTAIN,2019-05-26,33.808056,-105.738611,0.0
9,10663171.0,LONE MOUNTAIN,2019-05-27,33.808056,-105.738611,0.0
...,...,...,...,...,...,...
22275,11980587.0,CREEK 5,2020-12-25,33.402646,-117.266795,16.0
22276,11980587.0,CREEK 5,2020-12-26,33.402646,-117.266795,0.0
22277,11980587.0,CREEK 5,2020-12-27,33.402646,-117.266795,0.0
22278,11980587.0,CREEK 5,2020-12-28,33.402646,-117.266795,0.0


## 1. Figure out the start and end date we will use for each fire

In [ ]:
# first check what fraction of the fire lifetimes are 0 days
for item in incidents['Incident Number']:
    pts = fire_attributes['Number of VIIRS points'].values[fire_attributes['Incident Number']==item]
    #print(len(np.where(pts==0)[0]))
    print('The fire lifetime is ', len(np.where(pts!=0)[0])*100/len(pts), '% nonzero detections')

In [3]:
keep_days = []
for item in incidents['Incident Number']:
    pts = fire_attributes['Number of VIIRS points'].values[fire_attributes['Incident Number']==item]
    dys = fire_attributes['Current Day'].values[fire_attributes['Incident Number']==item]
    nonzero = np.where(pts!=0)[0]
    first_day = dys[nonzero[0]]
    last_day = dys[nonzero[len(nonzero)-1]]
    keep_days = np.append(keep_days, np.where((fire_attributes['Incident Number']==item)&(fire_attributes['Current Day']>=first_day)&(fire_attributes['Current Day']<=last_day))[0])

keep_days = [int(p) for p in keep_days]
print(len(keep_days))
fire_attributes = fire_attributes.iloc[keep_days, :]
fire_polygons = fire_polygons.iloc[keep_days, :]
fire_attributes = fire_attributes.reset_index(drop=True) #reset the indices beecause we dropped some
fire_polygons = fire_polygons.reset_index(drop=True) #reset the indices beecause we dropped some

fire_polygons

8161


,geometry
0,"POLYGON ((-112.75455 33.78632, -112.76012 33.7..."
1,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
2,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
3,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
4,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
...,...
8156,"POLYGON ((-112.11669 34.11898, -112.12564 34.1..."
8157,"POLYGON ((-117.03702 33.90632, -117.04484 33.9..."
8158,"POLYGON ((-117.27595 33.37881, -117.27495 33.3..."
8159,"POLYGON ((-117.29549 33.36558, -117.29713 33.4..."


## 2. Loop through the days and get the raw features (temp at different pressure levels, wind, pressure, precip, terrain info) and the labels (QFED PM2.5 from the next day)

In [20]:
ndays = 2
# dictionaries that track the variables, dimensions, times and levels
varis = {'/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.*.nc4':['TLML', 'QLML', 'SPEEDLML', 'PS'], 
        '/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.*.nc4': ['T', 'QV'],
         '/MERRA2/WESTUS_MERRA2_400.tavg1_2d_flx_Nx.*.nc4': ['PBLH', 'TCZPBL']} #don't have september for this!

#0=lat, 1=lon, 2=time, 3=level 
dims = {'/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.*.nc4':['lat', 'lon', 'time'], 
       '/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.*.nc4':['lat', 'lon', 'time', 'lev'],
        '/MERRA2/WESTUS_MERRA2_400.tavg1_2d_flx_Nx.*.nc4':['lat', 'lon', 'time']}

times = {'TLML':12, 'QLML':12, 'SPEEDLML':12, 'PS':12, 'T':12, 'QV':12, 'PBLH':12, 'TCZPBL':12}
levels = {'T':[500, 700], 'QV':[700]} 
file_suffix = '.nc4'

#pre-allocate the data frame
num_days = len(fire_attributes)
fire_features = gpd.GeoDataFrame({'Incident Number': np.nan*np.zeros(num_days), 'Fire Name': np.nan*np.zeros(num_days), 
                             'Current Day': np.nan*np.zeros(num_days), 'Lat Fire': np.nan*np.zeros(num_days), 
                             'Lon Fire': np.nan*np.zeros(num_days), 'Number of VIIRS points': np.nan*np.zeros(num_days),
                            'TLML_12Z_0': np.nan*np.zeros(num_days), 'QLML_12Z_0': np.nan*np.zeros(num_days),
                            'SPEEDLML_12Z_0': np.nan*np.zeros(num_days),'PS_12Z_0': np.nan*np.zeros(num_days),
                            'T_12Z_700mb_0': np.nan*np.zeros(num_days),'T_12Z_500mb_0': np.nan*np.zeros(num_days),
                            'QV_12Z_700mb_0': np.nan*np.zeros(num_days),'PBLH_12Z_0': np.nan*np.zeros(num_days), 
                            'TCZPBL_12Z_0': np.nan*np.zeros(num_days),'TLML_12Z_1': np.nan*np.zeros(num_days), 
                            'QLML_12Z_1': np.nan*np.zeros(num_days), 'SPEEDLML_12Z_1': np.nan*np.zeros(num_days),
                                  'PS_12Z_1': np.nan*np.zeros(num_days),'T_12Z_700mb_1': np.nan*np.zeros(num_days),
                                  'T_12Z_500mb_1': np.nan*np.zeros(num_days),'QV_12Z_700mb_1': np.nan*np.zeros(num_days),
                                  'PBLH_12Z_1': np.nan*np.zeros(num_days), 'TCZPBL_12Z_1': np.nan*np.zeros(num_days),  
                                  'geometry': np.nan*np.zeros(num_days)}) # pre-allocate the dataframe

#'fccs': np.nan*np.zeros(num_days),'slp': np.nan*np.zeros(num_days),'asp': np.nan*np.zeros(num_days), 
fire_features['Incident Number'] = fire_attributes['Incident Number'].values
fire_features['Fire Name'] = fire_attributes['Fire Name'].values
fire_features['Current Day'] = fire_attributes['Current Day'].values
fire_features['Lat Fire'] = fire_attributes['Lat Fire'].values
fire_features['Lon Fire'] = fire_attributes['Lon Fire'].values
fire_features['Number of VIIRS points'] = fire_attributes['Number of VIIRS points'].values
fire_features['geometry'] = fire_polygons['geometry'].values

#print(fire_features)
unique_days = np.unique(fire_features['Current Day'].values)
print(len(unique_days))
for dd in range(len(unique_days)):
    #print(unique_days[dd])
    for dyf in range(ndays):
        date=np.datetime64(unique_days[dd])+np.array(dyf, 'timedelta64[D]')
        print(date)
        date=str(date)
        yr = date[0:4]
        mo = date[5:7]
        dy = date[8:10]
        #print(yr, mo,dy)
        #find the polygons for the given day

        for i in varis : #loop over the file names
            #get the path to the file
            path = '/data2/lthapa/'+yr+i
            if '*' in path:
                path = path.replace('*', yr+mo+dy)
            if np.datetime64(date)>=np.datetime64('2020-09-01') and np.datetime64(date)<=np.datetime64('2020-09-30'):
                path = path.replace('400', '401')

            #load the file in as a netcdf dataset
            print(path)
            dat = nc.Dataset(path)
            ndims = len(dims[i])
            if ndims ==2: #lat and lon
                lat = dat[dims[i][0]][:]
                lon = dat[dims[i][1]][:]
            elif ndims == 3: #lat, lon, and time
                lat = dat[dims[i][0]][:]
                lon = dat[dims[i][1]][:]
                t = dat[dims[i][2]][:]
            elif ndims ==4: #lat,lon,time,and level
                lat = dat[dims[i][0]][:]
                lon = dat[dims[i][1]][:]
                t = dat[dims[i][2]][:]
                lev = dat[dims[i][3]][:]
            if len(lat.shape)==1:
                lon, lat = np.meshgrid(lon, lat)

            #calculate the areas of the lat/lon grid for the DATASET
            lat_corners, lon_corners = calculate_grid_cell_corners(lat, lon)
            inds = np.where(fire_features['Current Day']==unique_days[dd])[0]
            polys = fire_features['geometry'].iloc[inds].values
            print(inds)
            for p in range(len(inds)):
                #tic = time.time()
                #add a halo to the polygon
                poly = polys[p]
                poly_buff = poly.buffer(2) #buffer doesn't matter!
                # get the bounds of the buffered polygon
                w,s,e,n = poly_buff.bounds
                #print(w,s,e,n)
                s_ind = np.unique(np.argmin(np.abs(lat-s), axis=0))[0]
                n_ind = np.unique(np.argmin(np.abs(lat-n), axis=0))[0]
                w_ind = np.unique(np.argmin(np.abs(lon-w), axis=1))[0]
                e_ind = np.unique(np.argmin(np.abs(lon-e), axis=1))[0]


                #tic = time.time()
                overlaps = calculate_overlaps(lat,lon,lat_corners,lon_corners,s_ind,n_ind,w_ind,e_ind, poly) #calculate the overlap between the poly and the lat/lon grid
                #print(np.where(overlaps!=0))
                #toc = time.time()
                #print('Time elapsed for calculating overlaps for polygon:', toc-tic)
                for j in varis[i]: #loop over the variable names, which are lists
                    var = dat[j][:]
                    #print('the dimensions of', j, 'are', var.shape, 'and', j, 'has', ndims, 'dimensions')
                    #select the times and levels we need
                    if ndims ==2: #lat and lon
                        # calculate overlap
                        #print(var.shape)
                        var_weighted = np.multiply(var,overlaps)
                        var_ave = np.nansum(var_weighted)
                        name_in_table = j
                        fire_features.loc[inds[p],name_in_table] = var_ave

                    elif ndims == 3: #lat, lon, and time
                        #select the needed times (12Z)
                        if j in times.keys():
                            ind_time = np.argmin(np.abs(times[j]*60-t))
                            #filter the variable by time
                            var = var[ind_time, :, :]
                            #print(var.shape)
                            var_weighted = np.multiply(var,overlaps)
                            var_ave = np.nansum(var_weighted)
                            name_in_table = j+'_'+str(times[j])+'Z_'+str(dyf)
                            fire_features.loc[inds[p],name_in_table] = var_ave

                    elif ndims ==4: #lat,lon,time,and level
                        #select the needed times and levels
                        if j in times.keys():
                            ind_time = np.argmin(np.abs(times[j]*60-t))
                            #filter the variable by time
                            var = var[ind_time, :, :, :]
                            #filter by levels
                            if j in levels.keys(): #if there are levels, loop through them
                                for l in levels[j]:
                                    ind_lev = np.argmin(np.abs(l-lev))
                                    var_weighted = np.multiply(var[ind_lev, :, :], overlaps)
                                    var_ave = np.nansum(var_weighted)
                                    name_in_table = j+'_'+str(times[j])+'Z_'+str(l)+'mb_'+str(dyf)
                                    fire_features.loc[inds[p],name_in_table] = var_ave
                #toc = time.time()
                #print('Time to process polygon=',toc-tic)
            dat.close() 
fire_features

442
2019-02-17
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190217.nc4
[470]
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20190217.nc4
[470]
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.tavg1_2d_flx_Nx.20190217.nc4
[470]
2019-02-18
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190218.nc4
[470]
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20190218.nc4
[470]
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.tavg1_2d_flx_Nx.20190218.nc4
[470]
2019-02-18
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190218.nc4
[471]
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20190218.nc4
[471]
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.tavg1_2d_flx_Nx.20190218.nc4
[471]
2019-02-19
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190219.nc4
[471]
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20190219.nc4
[471]
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.tavg1_2d_flx_Nx.20190219.nc4
[471]
2019

[ 217 1174 1495]
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20190505.nc4
[ 217 1174 1495]
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.tavg1_2d_flx_Nx.20190505.nc4
[ 217 1174 1495]
2019-05-06
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190506.nc4
[ 217 1174 1495]
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20190506.nc4
[ 217 1174 1495]
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.tavg1_2d_flx_Nx.20190506.nc4
[ 217 1174 1495]
2019-05-06
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190506.nc4
[ 215  218 1103 1175 1496]
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20190506.nc4
[ 215  218 1103 1175 1496]
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.tavg1_2d_flx_Nx.20190506.nc4
[ 215  218 1103 1175 1496]
2019-05-07
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190507.nc4
[ 215  218 1103 1175 1496]
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20190507.nc4
[ 215  218 1103 1175 1496]


/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.tavg1_2d_flx_Nx.20190527.nc4
[38]
2019-05-28
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190528.nc4
[38]
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20190528.nc4
[38]
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.tavg1_2d_flx_Nx.20190528.nc4
[38]
2019-05-28
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190528.nc4
[39]
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20190528.nc4
[39]
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.tavg1_2d_flx_Nx.20190528.nc4
[39]
2019-05-29
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190529.nc4
[39]
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20190529.nc4
[39]
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.tavg1_2d_flx_Nx.20190529.nc4
[39]
2019-05-29
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190529.nc4
[  40 1426]
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20190529.nc4
[  40 1426]
/data2

/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.tavg1_2d_flx_Nx.20190610.nc4
[   2   54   79  487 1399 1438 1445 1546]
2019-06-11
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190611.nc4
[   2   54   79  487 1399 1438 1445 1546]
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20190611.nc4
[   2   54   79  487 1399 1438 1445 1546]
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.tavg1_2d_flx_Nx.20190611.nc4
[   2   54   79  487 1399 1438 1445 1546]
2019-06-11
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190611.nc4
[   3   55   80  488  645  646 1400 1439 1547]
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20190611.nc4
[   3   55   80  488  645  646 1400 1439 1547]
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.tavg1_2d_flx_Nx.20190611.nc4
[   3   55   80  488  645  646 1400 1439 1547]
2019-06-12
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190612.nc4
[   3   55   80  488  645  646 1400 1439 1547]
/data2/lthapa/2019/MERRA2/W

/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.tavg1_2d_flx_Nx.20190622.nc4
[  14  176  499 1411 1531 2168 2268 2280]
2019-06-23
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190623.nc4
[  14  176  499 1411 1531 2168 2268 2280]
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20190623.nc4
[  14  176  499 1411 1531 2168 2268 2280]
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.tavg1_2d_flx_Nx.20190623.nc4
[  14  176  499 1411 1531 2168 2268 2280]
2019-06-23
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190623.nc4
[  15  177  500 1177 1412 1532 2269 2281]
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20190623.nc4
[  15  177  500 1177 1412 1532 2269 2281]
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.tavg1_2d_flx_Nx.20190623.nc4
[  15  177  500 1177 1412 1532 2269 2281]
2019-06-24
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190624.nc4
[  15  177  500 1177 1412 1532 2269 2281]
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.ins

...

/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20201102.nc4
[3295 3672 3863 4146 4257 4565 4722 5388 5577 5828 5971 6947 7255 7348
 7475 7949 8101 8104 8126 8155]
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.tavg1_2d_flx_Nx.20201102.nc4
[3295 3672 3863 4146 4257 4565 4722 5388 5577 5828 5971 6947 7255 7348
 7475 7949 8101 8104 8126 8155]
2020-11-02
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20201102.nc4
[3296 3673 3864 4147 4258 4566 4723 5389 5578 5829 5972 6948 7256 7349
 7476 7950 8102 8105 8127 8145 8156]
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20201102.nc4
[3296 3673 3864 4147 4258 4566 4723 5389 5578 5829 5972 6948 7256 7349
 7476 7950 8102 8105 8127 8145 8156]
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.tavg1_2d_flx_Nx.20201102.nc4
[3296 3673 3864 4147 4258 4566 4723 5389 5578 5829 5972 6948 7256 7349
 7476 7950 8102 8105 8127 8145 8156]
2020-11-03
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20201103.nc4
[3296 3673

/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.tavg1_2d_flx_Nx.20201111.nc4
[4267 4575 5587 6957 7358 7485]
2020-11-12
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20201112.nc4
[4267 4575 5587 6957 7358 7485]
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20201112.nc4
[4267 4575 5587 6957 7358 7485]
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.tavg1_2d_flx_Nx.20201112.nc4
[4267 4575 5587 6957 7358 7485]
2020-11-12
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20201112.nc4
[4268 4576 5588 6958 7359 7486]
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20201112.nc4
[4268 4576 5588 6958 7359 7486]
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.tavg1_2d_flx_Nx.20201112.nc4
[4268 4576 5588 6958 7359 7486]
2020-11-13
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20201113.nc4
[4268 4576 5588 6958 7359 7486]
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20201113.nc4
[4268 4576 5588 6958 7359 7486]
/data2/lthapa/2020/MER

/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.tavg1_2d_flx_Nx.20201126.nc4
[4281 6971]
2020-11-26
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20201126.nc4
[4282 6972]
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20201126.nc4
[4282 6972]
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.tavg1_2d_flx_Nx.20201126.nc4
[4282 6972]
2020-11-27
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20201127.nc4
[4282 6972]
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20201127.nc4
[4282 6972]
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.tavg1_2d_flx_Nx.20201127.nc4
[4282 6972]
2020-11-27
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20201127.nc4
[4283 6973]
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20201127.nc4
[4283 6973]
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.tavg1_2d_flx_Nx.20201127.nc4
[4283 6973]
2020-11-28
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20201128.nc4
[4283 6973]
/data2/lthapa/2020/ME

/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20201213.nc4
[8157]
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.tavg1_2d_flx_Nx.20201213.nc4
[8157]
2020-12-14
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20201214.nc4
[8157]
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20201214.nc4
[8157]
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.tavg1_2d_flx_Nx.20201214.nc4
[8157]
2020-12-24
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20201224.nc4
[8158 8160]
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20201224.nc4
[8158 8160]
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.tavg1_2d_flx_Nx.20201224.nc4
[8158 8160]
2020-12-25
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20201225.nc4
[8158 8160]
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst3_3d_asm_Np.20201225.nc4
[8158 8160]
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.tavg1_2d_flx_Nx.20201225.nc4
[8158 8160]
2020-12-25
/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.in

,Incident Number,Fire Name,Current Day,Lat Fire,Lon Fire,Number of VIIRS points,TLML_12Z_0,QLML_12Z_0,SPEEDLML_12Z_0,PS_12Z_0,...,TLML_12Z_1,QLML_12Z_1,SPEEDLML_12Z_1,PS_12Z_1,T_12Z_700mb_1,T_12Z_500mb_1,QV_12Z_700mb_1,PBLH_12Z_1,TCZPBL_12Z_1,geometry
0,10662684.0,PAINTED WAGON,2019-04-08,33.786944,-112.753333,3.0,290.748108,0.004608,10.770707,91644.742188,...,295.369202,0.004046,2.791465,91247.414062,281.512573,261.920410,0.002269,253.785019,86.841118,"POLYGON ((-112.75455 33.78632, -112.76012 33.7..."
1,10663171.0,LONE MOUNTAIN,2019-06-09,33.808056,-105.738611,2.0,290.003723,0.003563,8.092171,81498.414062,...,281.681793,0.006098,10.386342,82444.875000,281.240356,262.869751,0.007128,456.560364,585.127686,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
2,10663171.0,LONE MOUNTAIN,2019-06-10,33.808056,-105.738611,0.0,281.681793,0.006098,10.386342,82444.875000,...,283.622253,0.009279,6.614388,81877.890625,283.966187,263.090515,0.005877,113.951096,175.440399,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
3,10663171.0,LONE MOUNTAIN,2019-06-11,33.808056,-105.738611,0.0,283.622253,0.009279,6.614388,81877.890625,...,284.536926,0.006591,12.682461,81796.609375,284.776917,267.068909,0.004644,314.892853,670.379883,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
4,10663171.0,LONE MOUNTAIN,2019-06-12,33.808056,-105.738611,0.0,284.536926,0.006591,12.682461,81796.609375,...,287.119812,0.010711,7.384966,81625.007812,284.941650,263.945251,0.006071,377.865326,376.500244,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8156,11979237.0,Trail,2020-11-02,34.147222,-112.117778,2.0,293.930328,0.003627,7.339819,91107.664062,...,292.495758,0.003988,8.028451,90745.757812,281.944702,259.631042,0.003883,64.640175,216.664398,"POLYGON ((-112.11669 34.11898, -112.12564 34.1..."
8157,11980186.0,SANDERSON,2020-12-13,33.889444,-117.070833,11.0,279.254852,0.001604,6.979411,90997.539062,...,279.923187,0.006894,4.845186,90440.250000,275.328552,256.890869,0.000052,163.259155,575.620300,"POLYGON ((-117.03702 33.90632, -117.04484 33.9..."
8158,11980587.0,CREEK 5,2020-12-24,33.402646,-117.266795,2.0,288.125366,0.001511,10.678336,99387.382812,...,287.946075,0.002882,5.147175,99128.710938,272.194824,258.612366,0.002691,328.705231,123.513199,"POLYGON ((-117.27595 33.37881, -117.27495 33.3..."
8159,11980587.0,CREEK 5,2020-12-25,33.402646,-117.266795,16.0,287.946075,0.002882,5.147175,99128.710938,...,286.868256,0.006895,1.817556,99087.804688,276.881714,259.645477,0.002607,63.415359,78.658180,"POLYGON ((-117.29549 33.36558, -117.29713 33.4..."


In [21]:
dataset = nc.Dataset('/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.tavg1_2d_flx_Nx.20190109.nc4')
print(dataset)
print(dataset['TCZPBL'][:])

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    History: Fri Jul 30 13:17:37 2021: ncks -d lon,-125.0,-101.0 -d lat,31.0,49.0 -v lat,lon,PBLH,TCZPBL MERRA2_400.tavg1_2d_flx_Nx.20190109.nc4 WESTUS_MERRA2_400.tavg1_2d_flx_Nx.20190109.nc4
Original file generated: Fri Feb  1 22:29:13 2019 GMT
    Comment: GMAO filename: d5124_m2_jan10.tavg1_2d_flx_Nx.20190109.nc4
    Filename: MERRA2_400.tavg1_2d_flx_Nx.20190109.nc4
    Conventions: CF-1
    Institution: NASA Global Modeling and Assimilation Office
    References: http://gmao.gsfc.nasa.gov
    Format: NetCDF-4/HDF-5
    SpatialCoverage: global
    VersionID: 5.12.4
    TemporalRange: 1980-01-01 -> 2016-12-31
    identifier_product_doi_authority: http://dx.doi.org/
    ShortName: M2T1NXFLX
    GranuleID: MERRA2_400.tavg1_2d_flx_Nx.20190109.nc4
    ProductionDateTime: Original file generated: Fri Feb  1 22:29:13 2019 GMT
    LongName: MERRA2 tavg1_2d_flx_Nx: 2d,1-Hourly,Time-Averaged,Single-Level,Ass

In [21]:
fire_features
therest = fire_features[['Incident Number', 'Fire Name', 'Current Day', 'Lat Fire', 'Lon Fire', 'Number of VIIRS points','TLML_12Z_0', 'QLML_12Z_0','SPEEDLML_12Z_0','PS_12Z_0','T_12Z_700mb_0','T_12Z_500mb_0','QV_12Z_700mb_0','PBLH_12Z_0', 'TCZPBL_12Z_0','TLML_12Z_1', 'QLML_12Z_1', 'SPEEDLML_12Z_1','PS_12Z_1','T_12Z_700mb_1','T_12Z_500mb_1','QV_12Z_700mb_1','PBLH_12Z_1', 'TCZPBL_12Z_1']]
print(therest)
therest.to_csv('fire_features_merra.csv', index=False)

      Incident Number      Fire Name Current Day   Lat Fire    Lon Fire  \
0          10662684.0  PAINTED WAGON  2019-04-08  33.786944 -112.753333   
1          10663171.0  LONE MOUNTAIN  2019-06-09  33.808056 -105.738611   
2          10663171.0  LONE MOUNTAIN  2019-06-10  33.808056 -105.738611   
3          10663171.0  LONE MOUNTAIN  2019-06-11  33.808056 -105.738611   
4          10663171.0  LONE MOUNTAIN  2019-06-12  33.808056 -105.738611   
...               ...            ...         ...        ...         ...   
8156       11979237.0          Trail  2020-11-02  34.147222 -112.117778   
8157       11980186.0      SANDERSON  2020-12-13  33.889444 -117.070833   
8158       11980587.0        CREEK 5  2020-12-24  33.402646 -117.266795   
8159       11980587.0        CREEK 5  2020-12-25  33.402646 -117.266795   
8160       11980949.0        CREEK 5  2020-12-24  33.407667 -117.269667   

      Number of VIIRS points  TLML_12Z_0  QLML_12Z_0  SPEEDLML_12Z_0  \
0                        3.

## Helper Functions

In [10]:
#this function calculates the area of each grid in a given grid cell
# LAT and LON should be 2d
def calculate_grid_cell_corners(LAT, LON):
    #we will assume the very edges of the polygons don't touch the boundary of the domain
    lat_corners = (LAT[0:(lat.shape[0]-1),  0:(lat.shape[1])-1] + LAT[1:(lat.shape[0]), 1:(lat.shape[1])])/2
    lon_corners = (LON[0:(lat.shape[0]-1),  0:(lat.shape[1])-1] + LON[1:(lat.shape[0]), 1:(lat.shape[1])])/2
    return lat_corners, lon_corners
    """for ii in range(1,LAT.shape[0]-1):
        for jj in range(1, LAT.shape[1]-1):
            cell_lat = LAT[ii, jj]
            cell_lon = LON[ii, jj]
            SW_lat[ii,jj] = LAT[ii,jj] + (LAT[ii-1,jj-1] - LAT[ii,jj])/2
            SW_lon[ii,jj] = LON[ii,jj] + (LON[ii-1,jj-1] - LON[ii,jj])/2
            SE_lat[ii,jj] = LAT[ii,jj] + (LAT[ii-1,jj-1] - LAT[ii,jj])/2
            SE_lon[ii,jj] = LON[ii,jj] + (LON[ii,jj] - LON[ii-1,jj-1])/2
            NW_lat[ii,jj] = LAT[ii,jj] + (LAT[ii,jj] - LAT[ii-1,jj-1])/2
            NW_lon[ii,jj] = LON[ii,jj] + (LON[ii-1,jj-1] - LON[ii,jj])/2
            NE_lat[ii,jj] = LAT[ii,jj] + (LAT[ii,jj] - LAT[ii-1,jj-1])/2
            NE_lon[ii,jj] = LON[ii,jj] + (LON[ii,jj] - LON[ii-1,jj-1])/2
    return  SW_lat, SW_lon, SE_lat, SE_lon, NW_lat, NW_lon, NE_lat, NE_lon"""

In [11]:
# this function calculates the overlap between the given shape (fire_shape) and given grid (LAT, LON). 
#LAT and LON should be 2D

def calculate_overlaps(lat, lon, lat_corners, lon_corners, lat_start, lat_stop, lon_start, lon_stop, fire_shape):
    overlaps = np.zeros(lat.shape)
    # buffer the fire shape so it has a non zero area
    if fire_shape.area ==0:
        fire_shape = fire_shape.buffer(0.01)
    # find the intersection between the first day's polygon and the  grid
    for ii in range(lat_start, lat_stop):
        for jj in range(lon_start, lon_stop):
            #print('ctr', lat[ii,jj], lon[ii,jj])
            """ SW_lat = lat_corners[ii-1, jj-1]
            SW_lon = lon_corners[ii-1, jj-1]
            SE_lat = lat_corners[ii-1, jj-1]
            SE_lon = lon_corners[ii-1, jj]
            NW_lat = lat_corners[ii, jj-1]
            NW_lon = lon_corners[ii, jj-1]
            NE_lat = lat_corners[ii, jj]
            NE_lon = lon_corners[ii, jj]"""
            poly_cell = Polygon([(lon_corners[ii, jj-1],lat_corners[ii, jj-1]), (lon_corners[ii, jj],lat_corners[ii, jj]), (lon_corners[ii-1, jj],lat_corners[ii-1, jj-1]), (lon_corners[ii-1, jj-1], lat_corners[ii-1, jj-1])])
            overlap_cell = fire_shape.intersection(poly_cell)
            #overlaps are the fraction of the POLYGON in each grid cell
            overlaps[ii,jj] = overlap_cell.area/fire_shape.area
    
    return overlaps